In [ ]:
import pandas
import matplotlib.pyplot as plt
dataset = pandas.read_csv('Rain_data_manipulated.csv', engine='python')
dataset.head()

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import numpy as np

In [ ]:
Labels=np.array(dataset['RainTomorrow'])

In [ ]:
Samples=np.array(dataset[['RainToday','Pressure','Humidity','Temp','WindVelocity','Brightness']])

In [ ]:
#*************************Do normalization on samples. Only Non-discrete columns.*************************************

In [ ]:
import numpy as np
from keras.layers import LSTM
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Flatten, Dropout, Activation, Lambda, Permute, Reshape
import json
import scipy
from keras.callbacks import Callback, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score
from sklearn.metrics import confusion_matrix
import tensorflow as tf

In [ ]:
def weather_predictor(X_train,Y_train,hparam):
    
    model = Sequential()

    input_shape=(X_train.shape[1],X_train.shape[2])

    model.add(LSTM(units=30,return_sequences=True,input_shape=input_shape))
    model.add(Dropout(0.2))

    model.add(LSTM(units=30))
    model.add(Dropout(0.2))
    model.add(Dense(unit=20))
    model.add(Dense(unit=10))
    model.add(Dense(unit=4))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(optimizer='adam', loss="binary_crossentropy",metrics=['accuracy'])
    trainer=model.fit(X_train, y_train, epochs=hparam.epoch,batch_size=hparam.batch,validation_split=hparam.eval_split)
    
    plt.plot(trainer.history['accuracy'])
    plt.plot(trainer.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'val'], loc='upper left')
    plt.show()
    plt.plot(trainer.history['loss'])
    plt.plot(trainer.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'val'], loc='upper left')
    plt.show()

    return model

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(Samples_normalized,Labels_normalized,test_size=0.2,random_state=42)

In [4]:
class hparam_config():
      def __init__(self, epoch,batch,eval_split):
        self.epoch=epoch
        self.batch=batch
        self.eval_split=eval_split

param_list=hparam_config(epoch=10,batch=100,eval_split=0.2)

In [ ]:
wraps=weather_predictor(X_train,Y_train,param_list)

In [ ]:
Y_predproba=wraps.predict(X_test,Y_test)

In [1]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [ ]:
lr_auc = roc_auc_score(y_test_final,Y_predproba)
print(lr_auc)
lr_fpr, lr_tpr, _ = roc_curve(Y_test, y_predproba)
plt.plot(lr_fpr, lr_tpr, marker='.', label='Wraps')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score
Y_pred=Y_predproba>0.5
accuracy_score(Y_test,Y_pred)

In [ ]:
import scikitplot as skplt
skplt.metrics.plot_confusion_matrix(
    Y_test, 
    Y_pred,
    figsize=(12,12))

In [ ]:
from keras.models import save_model
wraps.save('Wraps_LSTM_predictor.h5')  #*****************Run this cell only for improved models.